In [3]:
pip install transformers

In [4]:
pip install datasets

In [5]:
pip install torch

In [6]:
import transformers, datasets, torch
from datasets import load_dataset

In [7]:
dataset_tamil = load_dataset("SPRINGLab/IndicVoices-R_Tamil", split = 'train', streaming = True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
dataset_hindi = load_dataset("SPRINGLab/IndicVoices-R_Hindi", split = 'train', streaming = True)

In [9]:
dataset_bengali = load_dataset("SPRINGLab/IndicVoices-R_Bengali", split = 'train', streaming = True)

In [10]:
dataset_bengali

IterableDataset({
    features: ['text', 'lang', 'samples', 'verbatim', 'normalized', 'speaker_id', 'scenario', 'task_name', 'gender', 'age_group', 'job_type', 'qualification', 'area', 'district', 'state', 'occupation', 'utterance_pitch_mean', 'utterance_pitch_std', 'snr', 'c50', 'speaking_rate', 'cer', 'duration', 'audio'],
    num_shards: 16
})

In [11]:
dataset_bengali.features

{'text': Value(dtype='string', id=None),
 'lang': ClassLabel(names=['bn'], id=None),
 'samples': Value(dtype='int64', id=None),
 'verbatim': Value(dtype='string', id=None),
 'normalized': Value(dtype='string', id=None),
 'speaker_id': Value(dtype='string', id=None),
 'scenario': ClassLabel(names=['Extempore', 'Read'], id=None),
 'task_name': Value(dtype='string', id=None),
 'gender': ClassLabel(names=['Female', 'Male'], id=None),
 'age_group': ClassLabel(names=['18-30', '30-45', '45-60', '60+'], id=None),
 'job_type': ClassLabel(names=['Blue Collar', 'Student', 'Unemployed', 'White Collar'], id=None),
 'qualification': ClassLabel(names=['No Schooling', 'Post Grad + PhD', 'Undergrad and Grad.', 'Upto 12th'], id=None),
 'area': ClassLabel(names=['Rural', 'Urban'], id=None),
 'district': Value(dtype='string', id=None),
 'state': ClassLabel(names=['West Bengal'], id=None),
 'occupation': Value(dtype='string', id=None),
 'utterance_pitch_mean': Value(dtype='float64', id=None),
 'utterance_p

In [12]:
for i , k in enumerate(dataset_hindi):
  if i == 14:
    print(k['speaker_id'])
    break

S4259027400309283


In [13]:
#Only taking 500 rows
subset = []
for i, example in enumerate(dataset_tamil):
    if i >= 500:
        break
    subset.append(example)

In [14]:
for i, example in enumerate(dataset_hindi):
    if i >= 500:
        break
    subset.append(example)

In [15]:
for i, example in enumerate(dataset_bengali):
    if i >= 500:
        break
    subset.append(example)

In [16]:
len(subset)#3 languages 500 rows each hence 1500

1500

In [17]:
subset[0]

{'text': 'நான் எதனா வந்துட்டு தப்பாக சொல்லியிருந்தா என்னை மன்னிச்சுடுங்க ஆனால் வந்துட்டு இனிமேல் யாரை கூப்பிட்டாலுமே கரெக்ட் டைமுக்கு போய் அவங்கள கொண்டு போய் விட்டுருங்க ஏன்னால் அவங்களுக்கு அதுவே வாழ்க்கையாக கூட அமையலாம் சரிங்களா அண்ணா ஓகே அண்ணா',
 'lang': 0,
 'samples': 498154,
 'verbatim': 'நா எதுனா வந்துட்டு தப்பா சொல்லிருந்தா என்ன மன்னிச்சுருங்க ஆனா வந்துட்டு இனிமேல் யார கூப்பிடாலுமே கரெக்ட் டைமுக்கு போயி அவுங்கள கொண்டு போயி விட்ருங்க ஏன்னா அவுங்களுக்கு அதுவே வாழ்க்கையா கூட அமையலாம் சேரிங்களா அண்ணா ஓகே அண்ணா',
 'normalized': 'நான் எதனா வந்துட்டு தப்பாக சொல்லியிருந்தா என்னை மன்னிச்சுடுங்க ஆனால் வந்துட்டு இனிமேல் யாரை கூப்பிட்டாலுமே கரெக்ட் டைமுக்கு போய் அவங்கள கொண்டு போய் விட்டுருங்க ஏன்னால் அவங்களுக்கு அதுவே வாழ்க்கையாக கூட அமையலாம் சரிங்களா அண்ணா ஓகே அண்ணா',
 'speaker_id': 'S4257183500382558',
 'scenario': 0,
 'task_name': 'Ola/Uber Prompts',
 'gender': 0,
 'age_group': 0,
 'job_type': 1,
 'qualification': 1,
 'area': 0,
 'district': 'Salem',
 'state': 0,
 'occupation': 'STUDENT',

In [18]:
subset[514]['speaker_id']

'S4259027400309283'

In [19]:
import random

random.seed(42)
random.shuffle(subset)#shuffling the dataset

In [22]:
subset[514]['speaker_id']

'S4256570100331491'

In [23]:
#loading wav2vec model to extract features from audio files.
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from tqdm import tqdm
# Load the Wav2Vec2 processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
call_count=0
def extract_wav2vec_features(batch):
    global call_count
    call_count+=1
    print(call_count)
    inputs = processor(batch["audio"]["array"], sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs,output_hidden_states=True)
        # Pass through model and extract hidden states
    # Extract 9th and 11th layer hidden states
    layer_9_hidden_states = outputs.hidden_states[9]
    layer_11_hidden_states = outputs.hidden_states[11]

    # Average the two layers
    averaged_features = torch.mean(torch.stack([layer_9_hidden_states, layer_11_hidden_states]), dim=0)

    return averaged_features


In [25]:
extract_wav2vec_features(subset[0]).shape

1


torch.Size([1, 1622, 768])

In [26]:
#Number of parameters in the model

# Count the parameters
num_params = sum(p.numel() for p in model.parameters())

# Convert to millions and round to two decimal places
num_params_millions = round(num_params / 1e6, 2)

print(f"Total parameters (in millions): {num_params_millions}")

Total parameters (in millions): 94.37


In [27]:
### checking the model
print(model)

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
  

In [28]:
###checking the 7th and 12th layer

call_count=0
def extract_wav2vec_features(batch):
    global call_count
    call_count+=1
    print(call_count)
    inputs = processor(batch["audio"]["array"], sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs,output_hidden_states=True)
        # Pass through model and extract hidden states
    # Extract 9th and 11th layer hidden states
    layer_7_hidden_states = outputs.hidden_states[7]
    layer_12_hidden_states = outputs.hidden_states[12]

    # Average the two layers
    averaged_features = torch.mean(torch.stack([layer_7_hidden_states, layer_12_hidden_states]), dim=0)

    return averaged_features

In [29]:
extract_wav2vec_features(subset[0])

1


tensor([[[ 7.2330e-02, -2.9224e-02, -7.9637e-02,  ..., -6.4877e-03,
           2.1220e-02, -5.3084e-02],
         [ 1.1357e-01, -3.8196e-02, -6.1731e-02,  ...,  2.0457e-02,
           1.2855e-02, -2.4083e-02],
         [ 1.1619e-01, -4.9521e-02, -8.5681e-02,  ...,  1.7221e-02,
          -1.7617e-04,  1.8520e-02],
         ...,
         [ 1.0888e-02, -1.8063e-02,  1.5697e-01,  ..., -1.9450e-01,
           1.3999e-01, -7.9198e-02],
         [ 5.9432e-02, -1.7651e-02,  1.7657e-01,  ..., -1.3508e-01,
           1.4229e-01, -6.5013e-02],
         [ 5.0563e-02, -1.6680e-02,  1.8727e-01,  ..., -1.2202e-01,
           1.5047e-01, -6.9425e-02]]])

In [30]:
for i , k in enumerate(dataset_tamil):
  if i == 24:
    print(k['gender'])
    break

0


In [31]:
for i , k in enumerate(dataset_tamil):
  if i == 24:
    print(k)
    break

{'text': 'அது இல்லைனா சும்மா கட்டி மறு எண்ணைய் ஊற்றி சரி ஓகே இந்த இந்தளவுக்குதான் இது இருக்குது அப்படினு சொல்லி நீவி கை கால் நல்லா நீவி விட்டு கட்டு போட்டு இத்தனை நாளைக்கி கட்டு', 'lang': 0, 'samples': 561702, 'verbatim': 'அது இல்லனா சும்மா கட்டி மறு எண்ணை ஊத்தி செரி ஓகே இந்த இந்தளவுக்குதான் இது இருக்கு அப்டினு சொல்லி நீவி கை கால் நல்லாமே நீவி உட்டு கட்டு போட்டு இத்தன நாளக்கி கட்டு', 'normalized': 'அது இல்லைனா சும்மா கட்டி மறு எண்ணைய் ஊற்றி சரி ஓகே இந்த இந்தளவுக்குதான் இது இருக்குது அப்படினு சொல்லி நீவி கை கால் நல்லா நீவி விட்டு கட்டு போட்டு இத்தனை நாளைக்கி கட்டு', 'speaker_id': 'S4258334400348893', 'scenario': 0, 'task_name': 'Daily Life', 'gender': 0, 'age_group': 2, 'job_type': 0, 'qualification': 3, 'area': 0, 'district': 'Coimbatore', 'state': 0, 'occupation': 'Hand lumber', 'utterance_pitch_mean': 172.32358, 'utterance_pitch_std': 29.145819, 'snr': 62.4980964661, 'c50': 59.2988891602, 'speaking_rate': 13.1113862641, 'cer': 'tensor(0.0443)', 'duration': 12.7370208333, 'audio': {'p

In [32]:
subset[0]

{'text': 'শাড়ি তৈরি করে তাঁতের শাড়ি তৈরি করে ন্যায্য মূল্যে বিক্রি করে এবং আমাদের এখান থেকে অনেক মানুষ সেখানে শাড়ি কিনতে যার ফলে ন্যায্য মূল্য না পাওয়ার জন্য তাদের খুব সমস্যা হয়',
 'lang': 0,
 'samples': 477030,
 'verbatim': 'শাড়ি তৈরি করে তাঁতের শাড়ি তৈরি করে ন্যায্য মূল্যে বিক্রি করে এবং আমাদের এখান থেকে অনেক মানুষ সেখানে শাড়ি কিনতে যার ফলে ন্যায্য মূল্য না পাওয়ার জন্য তাদের খুব সমস্যা হয়',
 'normalized': 'শাড়ি তৈরি করে তাঁতের শাড়ি তৈরি করে ন্যায্য মূল্যে বিক্রি করে এবং আমাদের এখান থেকে অনেক মানুষ সেখানে শাড়ি কিনতে যার ফলে ন্যায্য মূল্য না পাওয়ার জন্য তাদের খুব সমস্যা হয়',
 'speaker_id': 'S4258014500311887',
 'scenario': 0,
 'task_name': 'District Specific ',
 'gender': 0,
 'age_group': 2,
 'job_type': 2,
 'qualification': 3,
 'area': 0,
 'district': 'Nadia',
 'state': 0,
 'occupation': 'House wife',
 'utterance_pitch_mean': 215.095,
 'utterance_pitch_std': 14.8906975,
 'snr': 55.088432312,
 'c50': 59.8574905396,
 'speaking_rate': 15.4386316319,
 'cer': 'tensor(0.0051)',
 'duration':

In [8]:
import random
import torch
import numpy as np
import gc
from datasets import load_dataset
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import joblib

# Set random seed for reproducibility
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

# Define datasets
languages = {
    "Tamil": ("SPRINGLab/IndicVoices-R_Tamil", 0),
    "Hindi": ("SPRINGLab/IndicVoices-R_Hindi", 1),
    "Bengali": ("SPRINGLab/IndicVoices-R_Bengali", 2)
}

# Load datasets and process
data_combined = []
for lang, (dataset_name, label) in languages.items():
    dataset = load_dataset(dataset_name, split="train", streaming=True)
    subset = dataset.take(500)

    for example in subset:
        example["lang"] = label  # Assign label
        data_combined.append(example)

# Shuffle the combined dataset
random.shuffle(data_combined)

In [11]:
print(type(data_combined), data_combined[0])

<class 'list'> {'text': 'শাড়ি তৈরি করে তাঁতের শাড়ি তৈরি করে ন্যায্য মূল্যে বিক্রি করে এবং আমাদের এখান থেকে অনেক মানুষ সেখানে শাড়ি কিনতে যার ফলে ন্যায্য মূল্য না পাওয়ার জন্য তাদের খুব সমস্যা হয়', 'lang': 2, 'samples': 477030, 'verbatim': 'শাড়ি তৈরি করে তাঁতের শাড়ি তৈরি করে ন্যায্য মূল্যে বিক্রি করে এবং আমাদের এখান থেকে অনেক মানুষ সেখানে শাড়ি কিনতে যার ফলে ন্যায্য মূল্য না পাওয়ার জন্য তাদের খুব সমস্যা হয়', 'normalized': 'শাড়ি তৈরি করে তাঁতের শাড়ি তৈরি করে ন্যায্য মূল্যে বিক্রি করে এবং আমাদের এখান থেকে অনেক মানুষ সেখানে শাড়ি কিনতে যার ফলে ন্যায্য মূল্য না পাওয়ার জন্য তাদের খুব সমস্যা হয়', 'speaker_id': 'S4258014500311887', 'scenario': 0, 'task_name': 'District Specific ', 'gender': 0, 'age_group': 2, 'job_type': 2, 'qualification': 3, 'area': 0, 'district': 'Nadia', 'state': 0, 'occupation': 'House wife', 'utterance_pitch_mean': 215.095, 'utterance_pitch_std': 14.8906975, 'snr': 55.088432312, 'c50': 59.8574905396, 'speaking_rate': 15.4386316319, 'cer': 'tensor(0.0051)', 'duration': 10.817

In [12]:
# Load Wav2Vec2 processor and model
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def extract_features(audio_data):
    """Extract features from Wav2Vec2 model with proper aggregation."""
    input_values = processor(
        audio_data,
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    )['input_values'].to(device)

    with torch.no_grad():
        outputs = model(input_values, output_hidden_states=True)
        layer_9 = outputs.hidden_states[9].cpu()
        layer_11 = outputs.hidden_states[11].cpu()

        # Average layers and time dimension
        combined = (layer_9 + layer_11) / 2.0
        features = torch.mean(combined, dim=1).squeeze().numpy()

    # Free GPU memory
    del outputs, layer_9, layer_11, input_values, combined
    torch.cuda.empty_cache()
    return features



In [14]:
# Process data in chunks and save separately
chunk_size = 100
features_filenames = []
labels_filenames = []
chunk_count = 0

features_list = []
labels_list = []
count = 0

for sample in tqdm(data_combined, desc="Extracting Features", total=len(data_combined)):
    try:
        audio_array = sample['audio']['array']
        label = sample['lang']

        features = extract_features(audio_array)
        features_list.append(features)
        labels_list.append(label)
        count += 1

        # Save chunks and reset
        if count % chunk_size == 0:
            features_fn = f"features_{chunk_count}.pkl"
            labels_fn = f"labels_{chunk_count}.pkl"

            joblib.dump(features_list, features_fn)
            joblib.dump(labels_list, labels_fn)

            features_filenames.append(features_fn)
            labels_filenames.append(labels_fn)

            features_list.clear()
            labels_list.clear()
            chunk_count += 1
            gc.collect()

    except Exception as e:
        print(f"Error processing sample: {e}")

# Save remaining data
if features_list:
    features_fn = f"features_{chunk_count}.pkl"
    labels_fn = f"labels_{chunk_count}.pkl"

    joblib.dump(features_list, features_fn)
    joblib.dump(labels_list, labels_fn)

    features_filenames.append(features_fn)
    labels_filenames.append(labels_fn)
    features_list.clear()
    labels_list.clear()
    chunk_count += 1

# Load all chunks
all_features = []
for fn in features_filenames:
    all_features.extend(joblib.load(fn))

all_labels = []
for fn in labels_filenames:
    all_labels.extend(joblib.load(fn))

# Convert to numpy arrays
X = np.vstack(all_features)
y = np.array(all_labels)

Extracting Features: 100%|██████████| 1500/1500 [05:15<00:00,  4.75it/s]


In [15]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Train and evaluate
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 74.33%


In [17]:
 #Create validation split from X_train
_, X_val, _, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train
)

# Predict using existing model
val_pred = clf.predict(X_val)
val_accuracy = accuracy_score(y_val, val_pred)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Validation Accuracy: 79.58%


In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into train, validation, and test sets
train_data, test_data = train_test_split(subset_with_features, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.1)

# Extract features and labels
def prepare_data(data):
    features = [item["features"] for item in data]
    labels = [item["label"] for item in data]
    return torch.tensor(features), torch.tensor(labels)


train_features, train_labels = prepare_data(train_data)
val_features, val_labels = prepare_data(val_data)
test_features, test_labels = prepare_data(test_data)